In [43]:
import tensorly as tl
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from tensorly.decomposition import symmetric_power_iteration

len_doc = 3 # number of words in each document; this is fake for now

def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
  
def svd_whiten(X, k):
    # X: matrix to decompose
    # k: rank approximation
    
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # rank k approximation
    U = U[:,:k]
    s = s[:k]
    Vt = Vt[:k,:]
    
    D_half = np.diag(np.sqrt(s))
    W = np.diag(1./np.sqrt(s)) @ U.T

    # so np.dot(np.dot(W.T, X),W) is the identity matrix
        
    return W , D_half, U


def rank_k_pseudoinverse(X, k):
    # pseudoinverse of rank k approximation of X
    U, s, Vt = np.linalg.svd(X, full_matrices=False)

    return np.linalg.pinv(U[:,:k] @ np.diag(s[:k]) @ Vt[:k,:])
        
def model_1():
    component_param = {0: np.array([0.31, 0.25, 0.44]   ),
                      1: np.array([0.1, 0.1, 0.8]),
                      2: np.array([0.9, 0.03, 0.07])}
    h_weights = np.array([0.1, 0.6, 0.3])
    return component_param, h_weights

def model_2():
    component_param = {0: np.array([.1, .1, .1, .3, .4]),
                      1: np.array([.15, 0.05, 0.8, 0 ,0]),
                      2: np.array([.2, 0.15, 0.3, .1, .25])}
    h_weights = np.array([0.1, 0.6, 0.3])
    return component_param, h_weights


def generate_data_fixed(data_size=10000):
    # num_components = num_rows = num_views = 3
    
    component_param, h_weights = model_2() # subject to change
    
    comp_label = np.arange(0,len(h_weights))  # there are num_components of components; zero-indexing
    observation_label = np.arange(0, len(component_param[0])) # assume for now all the range of observation are the same
    
    data = []
    for i in range(data_size):
        # choose a component
        choose = np.random.choice(comp_label, p = h_weights)
        datum = []
        
        # there are three words
        # each doc has length 30
        for _ in range(len_doc):
            datum.append(np.random.choice(observation_label, p = component_param[choose]))            
        data.append(tuple(datum))
    
    rank = len(h_weights)
    obs_size = len(component_param[0])
    
    return rank, obs_size, component_param, h_weights, data

### Generate some data

In [49]:
# rank is the number of mixture components
#   this is named rank because the observation matrix must have full column rank    
# obs_size is the cardinality of the observation random variable
#   right now it is assumed that every component has the same range
#   this is reasonable as the vocabulary size is the same accross different documents
    
rank, obs_size, cp, h, data = generate_data_fixed(data_size=5000)
len_data = len(data)
identity = np.eye(obs_size)

# dummy encoding
get_identity_col = lambda col_num: identity[:, col_num] 

# create word count vector for each document
c_ns = [np.sum(np.array(([get_identity_col(i) for i in datum])), axis=0) for datum in data]

### Estimate the second- and third-order moment

In [50]:
M_2f = np.zeros((obs_size,obs_size))

for i in range(obs_size):
    M_2f[i,i] = sum([(pow(c_ns[n][i],2)-c_ns[n][i])/(len_doc*(len_doc-1)) for n in range(len_data)])/len_data
    
for i in range(obs_size):
    for j in range(i+1,obs_size):        
        temp = sum([ ( c_ns[n][i]*c_ns[n][j] )/(len_doc*(len_doc-1)) for n in range(len_data)])/len_data
        M_2f[i,j] = M_2f[j,i] = temp

M_3f = np.zeros((obs_size,obs_size,obs_size))

for i in range(obs_size):
    M_3f[i,i,i] = sum([(pow(c_ns[n][i],3)-3*pow(c_ns[n][i],2)+2*c_ns[n][i])/(len_doc*(len_doc-1)*(len_doc-2)) for n in range(len_data)])/len_data
    
for i in range(obs_size):
    for j in range(i+1,obs_size):        
        temp = sum([(pow(c_ns[n][i],2)*c_ns[n][j]-c_ns[n][i]*c_ns[n][j])/(len_doc*(len_doc-1)*(len_doc-2)) for n in range(len_data)])/len_data
        M_3f[i,i,j] = M_3f[i,j,i] = M_3f[j,i,i] = temp
        
        temp2 = sum([(pow(c_ns[n][j],2)*c_ns[n][i]-c_ns[n][j]*c_ns[n][i])/(len_doc*(len_doc-1)*(len_doc-2)) for n in range(len_data)])/len_data
        M_3f[j,j,i] = M_3f[j,i,j] = M_3f[i,j,j] = temp2
                
for i in range(obs_size):
    for j in range(i+1,obs_size):
        for k in range(j+1,obs_size):        
            temp = sum([( c_ns[n][i]*c_ns[n][j]*c_ns[n][k] )/(len_doc*(len_doc-1)*(len_doc-2)) for n in range(len_data)])/len_data
            M_3f[i,j,k] = M_3f[i,k,j] = M_3f[j,i,k] = M_3f[j,k,i] = M_3f[k,i,j] = M_3f[k,j,i] = temp

### Whiten the moment tensors

In [51]:
W, D_half, U = svd_whiten(M_2f, rank) # a bit a cheating here by using rank variable
whitened_t01 = tl.tenalg.multi_mode_dot(M_2f, [W, W], modes=[0,1])
whitened_t012 = tl.tenalg.multi_mode_dot(M_3f, [W, W, W], modes=[0,1,2])
matprint( W @ M_2f @ W.T)

          1  1.12167e-16  4.75909e-16  
1.67349e-16            1  2.93986e-16  
2.78209e-16  2.33218e-16            1  


### Tensor decomposition

In [52]:
w = []
u = []
deflated = None
for i in range(rank):
    if deflated is None:        
        w_temp, u_temp, deflated = symmetric_power_iteration(whitened_t012)        
    else:
        w_temp, u_temp, deflated = symmetric_power_iteration(deflated)
    w.append(w_temp)
    u.append(u_temp)

recovered_h = np.array([pow(1/i,2) for i in w])
print("weights: ", recovered_h)
os = [w[index] * U @ D_half @ u_i for index, u_i in enumerate(u)]
print("Params: ")
for i in os:
    print(i)

weights:  [0.019 0.425 0.584]
Params: 
[ 0.142 -0.143  0.057  0.49   0.43 ]
[0.162 0.144 0.262 0.149 0.275]
[ 0.157  0.046  0.798 -0.003 -0.003]


### Asymmetric tensor power iteration

In [53]:
M_2_pinv = rank_k_pseudoinverse(M_2f, 3)
T = M_3f

w = []
u = []
os = []
deflated = None
for i in range(rank):
    u = np.random.rand(obs_size)
    u = np.divide(u, np.linalg.norm(u)) # randomly draw a vector from the unit sphere
    for _ in range(15):
        u = tl.tenalg.multi_mode_dot(T, [M_2_pinv @ u, M_2_pinv @ u], modes=[1,2])
        u = np.divide(u, np.linalg.norm(u))

    
    a=np.divide(u, np.sqrt(np.abs(np.dot(u, M_2_pinv @ u))))
    lambda1 = tl.tenalg.multi_mode_dot(T, [M_2_pinv @ a ,M_2_pinv@ a, M_2_pinv @ a], modes=[0,1,2])
    T = T -  abs(lambda1)*np.tensordot(np.tensordot(a,a,axes=0),a,axes=0)
    os.append(a/np.sum(a))
    w.append(pow(1/lambda1,2))


print("weights: ",np.array(w))
print("Params: ")
for i in os:
    print(i)

weights:  [0.019 0.425 0.584]
Params: 
[ 0.153 -0.126  0.054  0.515  0.403]
[0.166 0.144 0.263 0.15  0.276]
[ 0.156  0.047  0.802 -0.003 -0.003]
